# Parsing Data Into Standard Formats

> Note: this notebook uses [`scikit-learn`](https://scikit-learn.org/). Install it into your virtual environment using `pip install scikit-learn`.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from utils.u02_parsing import get_header, get_data, measurement_type, simplify_magnetic_df

# matplotlib "inline" magic command
# allows plots to be displayed in the notebook
%matplotlib inline

The functions answering the exercises from the Reading Files into DataFrames tutorial are available in [utils/u02_parsing.py](utils/u02_parsing.py). In this tutorial we'll be adding ZFCFC data to our list of files to parse. 

### A quick aside on a bug

In the course of making this tutorial I found some odd behavior with some .dat files that affected the `get_data()` function. Specifically, the `pd.read_csv()` function gave inconsistent behavior regarding the `sep='\t'` argument. Example below:

In [ ]:
def old_get_data(file_path: str | Path) -> pd.DataFrame:
    file_path = Path(file_path)
    skip_rows = len(get_header(file_path))
    df = pd.read_csv(file_path, sep="\t", skiprows=skip_rows)
    return df

In [ ]:
old_mvsh1 = old_get_data("data/mvsh1.dat")
old_mvsh1.head()

In [ ]:
old_zfcfc1 = old_get_data("data/zfcfc1.dat")
old_zfcfc1.head()

Thus, the `get_data()` function found in [utils/u02_parsing.py](utils/u02_parsing.py) has been modified. When incorrectly parsed with `sep='\t'`, the returned DatFrame has only one column. These .dat files can be correctly parsed by letting `pd.read_csv()` infer the separator (which, strangely, does not work for some .dat files which require the `sep='\t'` argument).

The new `get_data()` function is as follows:

```python
def get_data(file_path: str | Path) -> pd.DataFrame:
    file_path = Path(file_path)
    skip_rows = len(get_header(file_path))
    df = pd.read_csv(file_path, sep="\t", skiprows=skip_rows)
    # for unknown reasons, some .dat files parse incorrectly with sep="\t"
    # the incorrectly parsed df has only one column
    if df.shape[1] == 1:
        df = pd.read_csv(file_path, skiprows=skip_rows)
    return df
```

### Now back to your regularly scheduled content

At the top of the notebook we've imported a handful of functions which help us parse .dat files:

```python
from utils.u02_parsing import get_header, get_data, measurement_type, simplify_magnetic_df
```

Let's combine these into a single function `magnetic_df()` which takes a file path and returns a DataFrame with the magnetic data:

In [ ]:
def magnetic_df(file_path: str | Path) -> pd.DataFrame:
    df = get_data(file_path)
    return simplify_magnetic_df(df)

Now we'll read in some data files.

In [ ]:
mvsh1 = magnetic_df("data/mvsh1.dat")
mvsh2 = magnetic_df("data/mvsh2.dat")
mvsh3 = magnetic_df("data/mvsh3.dat")
zfcfc1 = magnetic_df("data/zfcfc1.dat")
zfcfc2 = magnetic_df("data/zfcfc2.dat")
zfcfc3 = magnetic_df("data/zfcfc3.dat")

Let's take a look at the data in these files. Plotting and the `matplotlib` library will be the topic of a future tutorial. Here's something to get you started.

Side note: [here's an article describing the `enumerate` function](https://realpython.com/python-enumerate/). In short, it's a way to iterate over a list while also keeping track of the index of the current item in the list.

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True)
for i, mvsh in enumerate([mvsh1, mvsh2, mvsh3]):
    axs[i].plot(mvsh.index, mvsh['temp'])
    axs[i].set_title(f"mvsh{i+1}")
axs[0].set_ylabel("Temperature (K)")
axs[1].set_xlabel("Measurement #")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True)
for i, zfcfc in enumerate([zfcfc1, zfcfc2, zfcfc3]):
    axs[i].plot(zfcfc.index, zfcfc['temp'])
    axs[i].set_title(f"zfcfc{i+1}")
axs[0].set_ylabel("Temperature (K)")
axs[1].set_xlabel("Measurement #")
plt.show()

When we get to data modeling, we'll want easy access to individual experiments. For example, we might want to look at M vs H data at 5 K for many samples, or we might want to look only at FC data at 100 Oe for many samples. Utlitmately we'll have to combine our knowledge of the potential experiments found in a .dat file with a combination of some parsing algorithms to figure out how to extract the data we want.

For the two separate problems (M vs H and ZFCFC data) the way I've plotted the data illustrates what our algorithms will be searching for. 
- In the case of M vs H data, the transition between different M vs H experiments within a single .dat file will be marked by a sudden change in the temperature. 
- In the case of ZFCFC data, the transition from ZFC to FC data will be marked by an interruption in otherwise monotonically increasing/decreasing temperature.

We won't be able to automatically parse everything. We'll have to set some rules that users follow when creating .dat files. For example, users probably shouldn't include both an M vs H measurement in the same file as a ZFCFC measurement. We'll write some rules in a later tutorial.

### M vs H parsing

Let's start by trying to separate `mvsh1` into a list of DataFrames where each DataFrame is a single M vs H experiment.

In [ ]:
fig, ax = plt.subplots()
ax.plot(mvsh1.index, mvsh1['temp'])
ax.set_xlabel("Measurement #")
ax.set_ylabel("Temperature (K)")
plt.show()

Unfortunately we can't just pick out the unique temperature values, because temperature fluctuations between data points mean that basically every measurement will be at a unique temperature.

In [ ]:
temps = mvsh1['temp'].unique()
temps

In [ ]:
len(temps)

The user clearly didn't intend to make measurements at 3619 different temperatures. We can see that the temperature is changing in a stepwise fashion. Let's try to find the stepwise changes in temperature and use those to separate the data into individual experiments.

There are several ways we can find the different sections of the data corresponding to each nominal temperature. One way would be to round the temperature values to the nearest integer (or half integer). The following examples use [the `pandas` `apply()` function](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) with a [lambda function](https://realpython.com/python-lambda/) to round the temperature values to the nearest quarter integer. The nearest quarter integer is necesarry if we want to be able to differentiate between nominal temperatures separated by 0.5 K.

In [ ]:
mvsh1['temp'].apply(lambda x: round(x/0.25)*0.25).unique()

In [ ]:
mvsh2['temp'].apply(lambda x: round(x/0.25)*0.25).unique()

As is shown with the `mvsh2` data, simply rounding to a set level of precision won't work, as the temperature of the magnetometer sometimes has greater variance at higher temperatures.

Knowing this, we can make a slightly more sophisticated rounding function, `round_temperature()`, which rounds temperatures based on the magnitude of the temperature value.

### Excercise 2.1

Write a function `round_temperature()` which takes a value `x` and rounds it to the nearest
- 0.25 K if `x` is less than 10 K
- 0.5 K if 10 < `x` < 50 K
- 1 K if `x` > 50 K.

In [ ]:
def round_temperature(x):
    # your code here
    ...

In [ ]:
assert round_temperature(2.45) == 2.5
assert round_temperature(8.70) == 8.75
assert round_temperature(11.3) == 11.5
assert round_temperature(299.7) == 300.0

### Exercise 2.2

Write a function `simple_find_unique_temperatures()` which takes a `pandas` `Series` (a single column of a `DataFrame`) and returns a list of unique temperatures within the data. Use the `round_temperature()` function you wrote in Exercise 2.1 to round the temperature values.

In [ ]:
def simple_find_unique_temperatures(data: pd.Series) -> list[float]:
    # your code here
    ...

In [ ]:
assert simple_find_unique_temperatures(mvsh1['temp']) == [2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 300.0]
assert simple_find_unique_temperatures(mvsh2['temp']) == [5.0, 300.0]

While the simple methods above have worked so far in `magnetopy`, `multi_cauchy`, and other projects, there are probably more robust alorithms we could use. [Clustering algorithms](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py) are a good place to start. [Here's a great video explaining DBSCAN](https://youtu.be/RDZUdRSDOok), which we'll be using next. 

[`DBSCAN` is in `scikit-learn`.](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) Before using it, we'll want to normalize our data. We can do this using the [`StandardScaler` class in `scikit-learn`.](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) `StandardScaler` will subtract the mean of each column and divide by the standard deviation of each column (or `Series`, in our case); the resulting data has a mean of 0 and standard deviation of 1. This is useful to do before clustering because it will make the data more amenable to clustering algorithms.

Spend some time with the following functions to understand how they work. You can go through each function line by line in a separate cell and print out the results of each step to see what's happening.

In [ ]:
def label_clusters(vals: pd.Series, eps: float = 0.01, min_samples: int = 10) -> np.ndarray:
    reshaped_vals = vals.values.reshape(-1, 1)
    scaler = StandardScaler()
    reshaped_normalized_vals = scaler.fit_transform(reshaped_vals)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    cluster_labels = dbscan.fit_predict(reshaped_normalized_vals)
    return cluster_labels

def unique_values(
    x: pd.Series, eps: float = 0.01, min_samples: int = 10
) -> list[float]:
    cluster_labels = label_clusters(x, eps=eps, min_samples=min_samples)
    unique_values = []
    for i in np.unique(cluster_labels):
        # average the values in each cluster
        unique_val = np.mean(x[cluster_labels == i])
        unique_val = round(unique_val, 1)
        unique_values.append(unique_val)
    return unique_values

Let's confirm the same behavior as with our simple methods above.

In [ ]:
unique_values(mvsh1['temp'])

In [ ]:
unique_values(mvsh2['temp'])

In [ ]:
unique_values(mvsh3['temp'])

We can also replot the M vs H data to see how the clustering algorithm has separated the data.

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True)
for i, mvshX in enumerate([mvsh1, mvsh2, mvsh3]):
    mvsh = mvshX.copy()
    mvsh['cluster'] = label_clusters(mvsh['temp'])
    axs[i].scatter(mvsh.index, mvsh['temp'], c=mvsh['cluster'])
axs[0].set_ylabel("Temperature (K)")
axs[1].set_xlabel("Measurement #")
plt.show()

### Exercise 2.3

Write a function `mvsh()` that takes in a magnetic dataframe and returns a dictionary of DataFrames, where the keys are the nominal temperatures of the measurements and the values are the DataFrames containing the data for each measurement.

In [ ]:
def mvsh(file_df: pd.DataFrame) -> dict[float, pd.DataFrame]:
    # your code here
    ...

In [ ]:
assert list(mvsh(mvsh1).keys()) == [2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 300.0]
assert list(mvsh(mvsh2).keys()) == [5.0, 300.0]
assert list(mvsh(mvsh3).keys()) == [5.0]
assert len(mvsh(mvsh1)[2.0]) == len(mvsh(mvsh1)[2]) == 1124
assert len(mvsh(mvsh1)[300]) == 563
assert len(mvsh(mvsh2)[300]) == 445
assert len(mvsh(mvsh3)[5]) == 141

### ZFCFC parsing

Let's turn our attention back to ZFCFC data. Our eventual goal is to write a function which can give us a dictionary of DataFrames where the keys are "zfc" and "fc" and the values are the DataFrames containing the data for each measurement.

Here's the data we're working with:

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True)
for i, zfcfc in enumerate([zfcfc1, zfcfc2, zfcfc3]):
    axs[i].plot(zfcfc.index, zfcfc['temp'])
    axs[i].set_title(f"zfcfc{i+1}")
axs[0].set_ylabel("Temperature (K)")
axs[1].set_xlabel("Measurement #")
plt.show()

Separating ZFC and FC from within a ZFCFC file requires finding the turnaround point. There are two cases we'll consider:
1. The ZFC is collected first with monotonic temperature increase, temperature is reset to the starting temperature, and the FC is collected with monotonic temperature increase.
2. The ZFC is collected first with monotonic temperature increase, then the FC is collected with monotonic temperature decrease.

The `.diff()` method will help us easily see what's going on.

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True)
for i, zfcfc in enumerate([zfcfc1, zfcfc2, zfcfc3]):
    zfcfc['diff'] = zfcfc['temp'].diff()
    zfcfc['diff'].iloc[0] = zfcfc['diff'].iloc[1]
    axs[i].plot(zfcfc.index, zfcfc['diff'])
    axs[i].set_xlim(175, 300)
axs[0].set_ylabel("Temperature (K)")
axs[1].set_xlabel("Measurement #")
plt.show()

Case #1 is easy to identify -- there will be one value in the `diff` column which is much larger than the others.

### Exercise 2.4

Write a function `find_outlier_indices()` which takes a `pandas` `Series` and an optional `threshold` argument (default value of 3) and returns a list of indices where the values are greater than `threshold` standard deviations from the mean. This is referred to as the [z-score or standard score](https://en.wikipedia.org/wiki/Standard_score). It's [available in `scipy`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.zscore.html) but write it yourself using `pandas` and/or `numpy`.

In [ ]:
def find_outlier_indices(x: pd.Series, threshold: float = 3) -> list[int]:
    # your code here
    ...

In [ ]:
assert find_outlier_indices(zfcfc1['temp'].diff()) == [252]
assert find_outlier_indices(zfcfc2['temp'].diff()) == [199]
assert find_outlier_indices(zfcfc3['temp'].diff()) == []

Case #2 requires finding the point at which the difference in temperature between data points is closest to zero. Note that the end of both the ZFC and FC measurements will have temperature profiles which slowly approach the final temperature. We'll assume that there's only one ZFCFC measurement per file (as in, one field per file), so the turnaround point will be somewhere in the middle.

In [ ]:
fig, ax = plt.subplots()
ax.plot(zfcfc3.index, zfcfc3['temp'].diff())
ax.set_xlim(230, 280) # comment out this line to see all of the data
ax.set_xlabel("Measurement #")
ax.set_ylabel("Temperature Difference (K)")

### Exercise 2.5

Write a function `find_turnaround_point()` which takes a magnetic dataframe (of the sort returned by `magnetic_df()`) containing data from a ZFCFC file and returns the index of the dataframe representing the first data point of the FC measurement. This should work for both cases #1 and #2.

In [ ]:
def find_temp_turnaround_point(df: pd.DataFrame) -> int:
    # your code here
    ...

In [ ]:
assert find_temp_turnaround_point(zfcfc1) == 252
assert find_temp_turnaround_point(zfcfc2) == 199
assert find_temp_turnaround_point(zfcfc3) == 256

### Exercise 2.6

Finally, write a function `zfcfc()` which takes a magnetic data frame and returns a dictionary of DataFrames, where the keys are "zfc" and "fc" and the values are the DataFrames containing the data for each measurement.

In [ ]:
def zfcfc(file_df: pd.DataFrame) -> dict[str, pd.DataFrame]:
    # your code here
    ...

In [ ]:
assert len(zfcfc(zfcfc1)['zfc']) == 252
assert len(zfcfc(zfcfc1)['fc']) == 252
assert len(zfcfc(zfcfc2)['zfc']) == 199
assert len(zfcfc(zfcfc2)['fc']) == 295
assert len(zfcfc(zfcfc3)['zfc']) == 256
assert len(zfcfc(zfcfc3)['fc']) == 257

### Exercise 2.7

One final thing for this tutorial -- write a function `experiment_type()` which takes a magnetic data frame and returns a string indicating the type of measurement. The string should be one of "mvsh", "zfcfc". Based on the assumptions we've made about how people are creating .dat files, we'll assume that if it's a dc measurement (VSM or DC scan) and there's only one field, it's a ZFCFC experiment. Otherwise, it's an M vs H experiment.

In [ ]:
def experiment_type(df: pd.DataFrame) -> str:
    # your code here
    ...

In [ ]:
assert experiment_type(mvsh1) == 'mvsh'
assert experiment_type(mvsh2) == 'mvsh'
assert experiment_type(mvsh3) == 'mvsh'
assert experiment_type(zfcfc1) == 'zfcfc'
assert experiment_type(zfcfc2) == 'zfcfc'
assert experiment_type(zfcfc3) == 'zfcfc'

## The bigger picture

Having standard formats makes the following steps (analysis, plotting, etc.) much easier. Interfaces are important in software development; they allow for modularity and more reusable code. The following plotting functions are made using the functions we've written in this tutorial.

In [ ]:
def plot_mvsh(file_df: pd.DataFrame, temperature: float | None = None):
    dfs = mvsh(file_df)
    if temperature:
        dfs = {temperature: dfs[temperature]}
    fig, ax = plt.subplots()
    for temp, df in dfs.items():
        ax.plot(df['field'] / 10000, df['moment'], label = f"{temp} K")
    ax.set_xlabel("Field (T)")
    ax.set_ylabel("Moment (emu)")
    ax.legend()
    return fig, ax

In [ ]:
fig, ax = plot_mvsh(mvsh1)
plt.show()

In [ ]:
fig, ax = plot_mvsh(mvsh2)
plt.show()

In [ ]:
fig, ax = plot_mvsh(mvsh3)
plt.show()